In [1]:
import torch
print(torch.__version__)


2.3.0+cu121


In [2]:
import torch
import torchvision
import torchaudio

print("Torch:", torch.__version__)
print("Torchvision:", torchvision.__version__)
print("Torchaudio:", torchaudio.__version__)



Torch: 2.3.0+cu121
Torchvision: 0.18.0+cu121
Torchaudio: 2.3.0+cu121


In [3]:
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import Dataset
import pandas as pd
import numpy as np
from PIL import Image

import zipfile

# Replace with the actual filename if it's different
zip_path = "fer-2013.zip"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall("fer2013_extracted")

print("✅ Extraction complete!")


✅ Extraction complete!


In [4]:
import os

def print_tree(startpath, prefix=""):
    for item in os.listdir(startpath):
        path = os.path.join(startpath, item)
        if os.path.isdir(path):
            print(f"{prefix}📁 {item}/")
            print_tree(path, prefix + "    ")
        else:
            print(f"{prefix}📄 {item}")

# Replace with your actual extraction folder
#print_tree("fer2013_extracted")


In [5]:
import os
import torch
from torchvision import datasets, transforms
from torch.utils.data import Dataset

from torchvision import datasets
from torch.utils.data import Dataset
import os

class ImageFolderStressWrapper(Dataset):
    def __init__(self, root_dir, transform=None):
        self.base_dataset = datasets.ImageFolder(root=root_dir, transform=transform)
        self.samples = []

        # Define mapping
        stressed_classes = {'fear', 'angry', 'sad', 'disgust'}
        not_stressed_classes = {'happy', 'neutral'}

        for img_path, label in self.base_dataset.samples:
            class_name = os.path.basename(os.path.dirname(img_path)).lower()

            if class_name in stressed_classes:
                stress_label = 1
            elif class_name in not_stressed_classes:
                stress_label = 0
            else:
                continue  # skip unknown/surprise/etc.

            self.samples.append((img_path, stress_label))

        self.transform = transform

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, stress_label = self.samples[idx]
        image = self.base_dataset.loader(img_path)
        if self.transform:
            image = self.transform(image)
        return image, stress_label


In [6]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

dataset = ImageFolderStressWrapper("fer2013_extracted/train", transform=transform)



from torch.utils.data import random_split, DataLoader

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_ds, val_ds = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=32)


In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

# Load VGG-16 pretrained on ImageNet
vgg16 = models.vgg16(pretrained=True)

# Replace final classifier layer (original: 4096 → 1000) with 2-class output
vgg16.classifier[6] = nn.Linear(4096, 2)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vgg16.to(device)


/home/jupyter-st124872/.local/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/jupyter-st124872/.local/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(vgg16.parameters(), lr=1e-4)
num_epochs = 5  # Increase to 20+ for better results


In [9]:
for epoch in range(num_epochs):
    vgg16.train()
    total_loss = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = vgg16(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs} - Training Loss: {total_loss:.4f}")

    # Validation
    vgg16.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = vgg16(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    acc = 100 * correct / total
    print(f"✅ Validation Accuracy: {acc:.2f}%")


OutOfMemoryError: CUDA out of memory. Tried to allocate 392.00 MiB. GPU 

In [ ]:
torch.save(vgg16.state_dict(), "vgg16_stress.pth")
print("✅ Trained model saved.")


In [ ]:
from torch.quantization import quantize_dynamic

# Reload your trained model structure
vgg16 = models.vgg16()
vgg16.classifier[6] = nn.Linear(4096, 2)
vgg16.load_state_dict(torch.load("vgg16_stress.pth", map_location="cpu"))
vgg16.eval()

# Quantize: Apply INT8 to Linear layers
vgg16_quant = quantize_dynamic(vgg16, {nn.Linear}, dtype=torch.qint8)
print("✅ Model quantized to INT8 (FC layers).")


In [ ]:
import numpy as np
import json
import os

os.makedirs("fpga_weights", exist_ok=True)
layer_metadata = []

layer_id = 0
for name, module in vgg16_quant.named_modules():
    if isinstance(module, (nn.Conv2d, nn.Linear)):
        weight = module.weight.detach().cpu().numpy()
        weight_file = f"fpga_weights/layer{layer_id}_{name.replace('.', '_')}_weight.txt"
        np.savetxt(weight_file, weight.flatten(), fmt='%d')

        layer_metadata.append({
            "layer_id": layer_id,
            "name": name,
            "type": type(module).__name__,
            "shape": list(weight.shape),
            "file": weight_file
        })
        layer_id += 1

# Save metadata
with open("fpga_weights/model_layers.json", "w") as f:
    json.dump(layer_metadata, f, indent=2)

print("✅ All weights exported and model_layers.json created.")


In [10]:
def txt_to_mif(input_file, output_file, depth, width=8):
    with open(input_file, 'r') as f:
        data = [line.strip() for line in f if line.strip()]

    with open(output_file, 'w') as mif:
        mif.write(f"WIDTH={width};\n")
        mif.write(f"DEPTH={depth};\n")
        mif.write("ADDRESS_RADIX=UNS;\n")
        mif.write("DATA_RADIX=DEC;\n")
        mif.write("CONTENT BEGIN\n")

        for i, value in enumerate(data):
            mif.write(f"{i} : {value};\n")

        mif.write("END;\n")



In [11]:
txt_to_mif(
    "fpga_weights/layer0_features_0_weight.txt",
    "fpga_weights/layer0_features_0_weight.mif",
    depth=9  # For a 3x3 conv layer
)


In [13]:
import os

folder = "verilog files"  # Change to your folder

for filename in os.listdir(folder):
    if filename.endswith(".txt"):
        base = os.path.splitext(filename)[0]
        new_name = base + ".v"
        os.rename(os.path.join(folder, filename), os.path.join(folder, new_name))

print("✅ All .txt files renamed to .v")


✅ All .txt files renamed to .v


In [15]:
import os

folder = "verilog files"  # Change to your folder

for filename in os.listdir(folder):
    if filename.endswith("simulate.txt"):
        base = os.path.splitext(filename)[0]
        new_name = base + ".do"
        os.rename(os.path.join(folder, filename), os.path.join(folder, new_name))

print("✅ All simulate.txt files renamed to simulate.do")

✅ All simulate.txt files renamed to simulate.do
